In [ ]:
!pip install openai python-dotenv fastapi json-repair openai uvicorn fastapi

In [2]:
import json
from openai import OpenAI

In [3]:
# Serverless 메뉴에서 GPU 빌리고나면 모델명 아래에 보이는 모델 id값. 아래의 값은 현재 만료된 값입니다.
RUNPOD_ENDPOINT_ID='yjzx280tpgztc5'

In [ ]:
# Setting 메뉴에서 api_key 발급 받은 후에 사용. 별도로 저장해놨다가 사용. 처음에만 보이기 때문에 기록 필수
import os
from dotenv import load_dotenv

load_dotenv("./.env")
api_key = os.getenv('RUNPOD_API_KEY')

In [5]:
client = OpenAI(
    api_key=api_key,
    base_url=f"https://api.runpod.ai/v2/{RUNPOD_ENDPOINT_ID}/openai/v1"
)

In [6]:
MODEL_NAME = "iamjoon/llama3-8b-finance-analyzer"

In [9]:
system_prompt = """
당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.
두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.
해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 '없음' 등을 적어서는 안 됩니다.

만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": False,
"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}

만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": True,
"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],
"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",
"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주는데 근거가 되었던 주요한 명사 키워드들을 파이썬 문자열 리스트 형태로 작성하십시오. 기술명, 회사명 등을 모두 포함합니다. 복합 명사 또한 허용합니다. 없다면 빈 리스트로 작성합시오."],
"negative_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],
"reason_for_negative_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",
"negative_keywords": ["부정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 부정적인 영향을 주는데 근거가 되었던 주요한 명사 키워드들을 파이썬 문자열 리스트 형태로 작성하십시오. 기술명, 회사명 등을 모두 포함합니다. 복합 명사 또한 허용합니다. 없다면 빈 리스트로 작성합시오."],
"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}
"""


user_prompt = """HUG 국민·우리은행 등 표준 PF 주관금융사로 재선정 주택도시보증공사 HUG 는 주택사업자의 원활한 자금조달을 지원하는 표준 프로젝트 파이낸싱 PF 과 후분양 표준 PF 보증의 주관금융사를 새로 선정했다고 4일 밝혔다. PF 보증이란 주택 건설 사업의 미래 현금수입과 사업성을 담보로 주택 사업자가 대출받는 토지비 등 사업비에 대한 주택사업금융의 원리금 상환을 책임지는 보증을 말한다. 후분양 PF 보증이란 주택 사업자가 주택의 일부를 후분양하는 조건으로 주택 건설 자금 대출금을 조달하는 경우에 원리금 상환을 책임지는 보증이다. HUG는 2014년 제도를 시행한 이후 표준 PF 보증 약 12조6000억원 후분양 표준 PF 보증 약 8600억원을 지원했다. 이번에 새로 선정된 주관금융사는 표준 PF의 경우 국민은행 부산은행 수협은행 우리은행 하나은행이다. 후분양 표준 PF는 부산은행 우리은행 수협은행이다. 권형택 HUG 사장은 “최근 원자재가격 급등 대출금리 인상 등 비용증가로 어려움을 겪는 주택사업자에게 저금리 금융지원을 통하여 비용을 경감시킬 수 있게 된 점을 뜻깊게 생각한다”며 “표준PF 후분양 표준PF 제도운영을 통하여 주택공급 확대를 통한 부동산 시장 안정 주거안정 지원에 큰 도움이 될 것으로 전망한다”고 밝혔다."""

In [10]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [11]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
    temperature=0,
    stop=["\n<|end_of_text|>", "<|eot_id|>"],
)

In [13]:
response

ChatCompletion(id='chatcmpl-672b0c9053424351a6dc401bde5482f8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"is_stock_related": true, "negative_impact_stocks": [], "negative_keywords": [], "positive_impact_stocks": ["국민은행", "부산은행", "수협은행", "우리은행", "하나은행"], "positive_keywords": ["표준 PF", "후분양 PF", "주택사업자", "자금조달", "금융지원", "비용경감", "저금리", "주택공급", "부동산 시장 안정", "주거안정"], "reason_for_negative_impact": "", "reason_for_positive_impact": "HUG 국민·우리은행 등 표준 PF 주관금융사 재선정을 통해 주택사업자에게 저금리 금융지원을 제공할 수 있게 되어, 주택공급 확대와 부동산 시장 안정에 긍정적인 영향을 줄 것으로 전망된다.", "summary": "HUG는 표준 PF와 후분양 PF의 주관금융사를 새로 선정하여 주택사업자에게 저금리 금융지원을 제공하고, 주택공급 확대를 통한 부동산 시장 안정과 주거안정 지원에 기여할 것으로 예상된다."}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1757157959, model='iamjoon/llama3-8b-finance-analyzer', object='chat.completion', service_tier=None, system_fingerprint=None, usage

In [17]:
response.choices[0].message.content

'{"is_stock_related": true, "negative_impact_stocks": [], "negative_keywords": [], "positive_impact_stocks": ["국민은행", "부산은행", "수협은행", "우리은행", "하나은행"], "positive_keywords": ["표준 PF", "후분양 PF", "주택사업자", "자금조달", "금융지원", "비용경감", "저금리", "주택공급", "부동산 시장 안정", "주거안정"], "reason_for_negative_impact": "", "reason_for_positive_impact": "HUG 국민·우리은행 등 표준 PF 주관금융사 재선정을 통해 주택사업자에게 저금리 금융지원을 제공할 수 있게 되어, 주택공급 확대와 부동산 시장 안정에 긍정적인 영향을 줄 것으로 전망된다.", "summary": "HUG는 표준 PF와 후분양 PF의 주관금융사를 새로 선정하여 주택사업자에게 저금리 금융지원을 제공하고, 주택공급 확대를 통한 부동산 시장 안정과 주거안정 지원에 기여할 것으로 예상된다."}'

In [18]:
import json_repair

decoded_object = json_repair.loads(response.choices[0].message.content)
decoded_object

{'is_stock_related': True,
 'negative_impact_stocks': [],
 'negative_keywords': [],
 'positive_impact_stocks': ['국민은행', '부산은행', '수협은행', '우리은행', '하나은행'],
 'positive_keywords': ['표준 PF',
  '후분양 PF',
  '주택사업자',
  '자금조달',
  '금융지원',
  '비용경감',
  '저금리',
  '주택공급',
  '부동산 시장 안정',
  '주거안정'],
 'reason_for_negative_impact': '',
 'reason_for_positive_impact': 'HUG 국민·우리은행 등 표준 PF 주관금융사 재선정을 통해 주택사업자에게 저금리 금융지원을 제공할 수 있게 되어, 주택공급 확대와 부동산 시장 안정에 긍정적인 영향을 줄 것으로 전망된다.',
 'summary': 'HUG는 표준 PF와 후분양 PF의 주관금융사를 새로 선정하여 주택사업자에게 저금리 금융지원을 제공하고, 주택공급 확대를 통한 부동산 시장 안정과 주거안정 지원에 기여할 것으로 예상된다.'}

In [19]:
decoded_object["positive_impact_stocks"]

['국민은행', '부산은행', '수협은행', '우리은행', '하나은행']

In [24]:
user_prompt = """SK하이닉스가 올해 2분기에도 글로벌 D램 시장 1위를 차지하며 2위 삼성전자와 격차를 벌렸습니다.

오늘(5일) 시장조사업체 옴디아에 따르면 올해 2분기 글로벌 D램 업계의 매출 규모는 전 분기보다 17.3% 증가한 309억 1,600만 달러(약 43조 원)로 집계됐습니다.

D램 계약 가격 상승과 고대역폭메모리(HBM) 출하량 증가에 따른 결과로 풀이됩니다.

전체 D램 시장 점유율에서는 HBM 지배력에 따라 업체 간 희비가 갈렸습니다.

옴디아는 SK하이닉스의 D램 점유율(매출 기준)이 지난 1분기 36.9%에서 2분기 39.5%로 더 늘어, 2분기 연속 삼성전자를 앞지르고 1위를 차지했다고 밝혔습니다.

같은 기간 삼성전자의 점유율은 34.4%에서 33.3%로 하락해 2위에 머물렀습니다.

양사 간 격차는 2.5%p(포인트)에서 6.2%p로 벌어졌습니다.

매출 규모도 올해 2분기 SK하이닉스는 122억 2,600만 달러, 삼성전자는 103억 달러로 19억 달러 이상 차이를 보였습니다.

SK하이닉스가 D램 점유율에서 삼성전자를 추월한 건 지난 1분기가 처음이었습니다.

1992년 D램 시장에서 삼성전자가 세계 1위를 차지한 이후 33년 만입니다.

또 다른 시장조사업체 트렌드포스도 2분기 SK하이닉스의 D램 시장 점유율이 38.7%를 기록해 1위를 유지했다고 밝혔습니다."""

In [25]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [26]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
    temperature=0,
    stop=["\n<|end_of_text|>", "<|eot_id|>"],
)

In [27]:
decoded_object = json_repair.loads(response.choices[0].message.content)
decoded_object

{'is_stock_related': True,
 'negative_impact_stocks': [],
 'negative_keywords': [],
 'positive_impact_stocks': ['SK하이닉스', '삼성전자'],
 'positive_keywords': ['SK하이닉스', '삼성전자', 'D램 시장 점유율', '매출 규모', '고대역폭메모리(HBM)'],
 'reason_for_negative_impact': '',
 'reason_for_positive_impact': 'SK하이닉스가 글로벌 D램 시장에서 1위를 차지하고 매출 규모가 증가함에 따라 긍정적인 영향을 받을 것으로 보입니다. 삼성전자의 점유율 하락과 매출 감소로 인해 SK하이닉스에 대한 투자 기회가 늘어날 수 있습니다.',
 'summary': 'SK하이닉스가 올해 2분기에도 글로벌 D램 시장 1위를 차지하며 매출 규모가 증가한 반면, 삼성전자는 점유율과 매출에서 하락세를 보였습니다. 이는 SK하이닉스에 대한 투자 기회가 확대될 수 있음을 시사합니다.'}